This is a program that use supervised machine learning to predict stock prices. 

The script uses regularized linear regression that takes a series of continuous stock prices (Highest in the day) to predict stock prices (highest in the day) given immediately after. The data used is Tesla's historical stock prices downloaded from https://finance.yahoo.com/quote/TSLA/history?p=TSLA

In [608]:
import numpy
import csv
from sklearn.utils import shuffle

In [609]:
# General Parameters:

filename = 'TSLA.csv'
lag = 15 # Using the past 15 prices to predict the next
iterations = 300 # Train with 300 iterations
threshold = 9e9 # Stop training when cost exceeds the threshold

alpha_list = [1, 0.1, 0.01] # Training step size
lam_list = [0.01, 0.001, 0.0001] # Regularier parameter

First, read the data from csv sheets. If using lag=15, the script organizes prices of 15 continuous dates into a training example, and uses the 16th price as label. 

In [610]:
# Data Management:
X = []
Y = []
with open(filename, 'r') as djiFile:
    dji = csv.reader(djiFile)
    next(dji)
    row_buff = []
    
    for row in dji:
        if len(row_buff) < lag:
            row_buff.append(float(row[2]))
        else:
            X.append(row_buff)
            row_buff = row_buff[1:]
            row_buff.append(float(row[2]))
            Y.append(numpy.array(float(row[2])))
    
X = numpy.array(X)
Y = numpy.array(Y)
num_examples = numpy.size(X,0)
num_features = numpy.size(X,1)

print("Dataset size: ")
print(numpy.size(X,0), numpy.size(X,1))
print(numpy.size(Y))


Dataset size: 
993 15
993


Since the stock prices dealing with can sometimes be very high. If using DJIA it could be around 20,000, the script use mean normalization to preprocess the data. A feature of 1.0 is also added to all examples.

The normalized data is then shuffled, in case the training is influenced by potential trends over larger time periods. 

The script then separates all examples into training set, validation set, and test set, in the ratio of 60%:20%:20%. 

In [611]:
# Mean Normalization:
X_norm = numpy.zeros((num_examples, num_features), dtype=float)
X_mean = numpy.zeros((1, num_features), dtype=float)
X_std = numpy.zeros((1, num_features), dtype=float)

for j in range(0, num_features):
    X_mean[0,j] = numpy.mean(X[:,j])
    X_std[0,j] = numpy.std(X[:,j])
    X_norm[:,j] = (X[:,j] - X_mean[0,j]) / X_std[0,j]
    
Y_norm = (Y - numpy.mean(Y)) / numpy.std(Y)

# Add first feature as ones:
col_first = numpy.ones([numpy.size(X,0), 1])
X_norm = numpy.hstack((col_first, X_norm))
num_features += 1

# Shuffle data:
X_norm, Y_norm = shuffle(X_norm, Y_norm)

In [612]:
# Data separated into sets:
Xtrain = X_norm[0 : round(num_examples*0.6)]
Xval = X_norm[round(num_examples*0.6) : round(num_examples*0.8)]
Xtest = X_norm[round(num_examples*0.8) :]

Ytrain = Y_norm[0 : round(num_examples*0.6)]
Yval = Y_norm[round(num_examples*0.6) : round(num_examples*0.8)]
Ytest = Y_norm[round(num_examples*0.8) :]

size_train = numpy.size(Xtrain, 0)
size_val = numpy.size(Xval, 0)
size_test = numpy.size(Xtest, 0)

print("Training set size = ", numpy.size(Xtrain,0), numpy.size(Xtrain,1))
print("Validation set size = ", numpy.size(Xval,0))
print("Test set size = ", numpy.size(Xtest,0))
print("\nFirst three training examples: ")
print(Xtrain[0:3])
print("\nFirst three labels: ")
print(Ytrain[0:3])

Training set size =  596 16
Validation set size =  198
Test set size =  199

First three training examples: 
[[ 1.          2.68377766  2.21098085  1.88764249  2.14164381  2.44340912
   2.17214968  1.99619122  1.75412677  1.49786913  1.49135436  1.39173772
   1.08003592  1.11391572  0.5750608   0.45521856]
 [ 1.          0.16539199  0.21084699  0.15973434  0.12759969  0.11139001
   0.06530273 -0.00880898 -0.04711521 -0.05165879 -0.09998273 -0.09542066
  -0.10504283 -0.08734699 -0.04638823 -0.05745645]
 [ 1.          2.7575316   2.57421032  2.52989499  2.64333594  2.52521833
   2.48810754  2.44415086  2.38589824  2.3514499   2.27276091  2.215813
   2.21644523  2.46548686  2.46239572  2.36454192]]

First three labels: 
[ 0.1476996  -0.0505277   2.32169082]


The training set will output a parameter for every alpha and lambda. The training iterations for certain alpha and lambda will be skipped if the cost increases exceeding the threshold.

The validation set is used to find optimal alpha and lambda. 

In [613]:
# Validation set up:
param_all = numpy.zeros((num_features, len(alpha_list), len(lam_list)))
    # Collection of all parameters learned for every alpha and lambda combination
cost_opt = 999999
alpha_opt_index = 0;
lam_opt_index = 0;

In [614]:
# Train the model:

for a in range(0, len(alpha_list)):
    alpha = alpha_list[a]
    
    for l in range(0, len(lam_list)):
        lam = lam_list[l]
        
        param = numpy.zeros((num_features, 1), dtype=float)
        cost = 0
        print("Training with alpha = ", alpha, " and lambda = ", lam)
        
        for r in range(0, iterations):
            param_temp = numpy.zeros(numpy.size(param))

            for j in range(0, num_features): 

                grad = 0
                for i in range(0, size_train):
                    grad += (Xtrain[i]@param - Ytrain[i]) * Xtrain[i,j]
                grad = (1/size_train) * grad
                if j != 0:
                    grad += (lam/size_train) * param[j]  
                param_temp[j] = param[j] - alpha * grad

            for j in range(0, num_features): 
                param[j] = param_temp[j]

            # Cost:
            sumSqrError = 0
            for i in range(0, size_train):
                sumSqrError += (Xtrain[i] @ param - Ytrain[i]) ** 2
            sumSqrParam = 0
            for j in range(1, num_features):
                sumSqrParam += param[j] ** 2
            cost = (1/(2*size_train)) * sumSqrError + lam * sumSqrParam
            #print("Iteration ", r+1, ", cost = ", cost, end='\n')
            
            if cost > threshold:
                print(" Skipped")
                break
            
        print(" cost = ", cost, end='\n')
        for j in range(0, num_features):
            param_all[j,a,l] = param[j]

Training with alpha =  1  and lambda =  0.01
 Skipped
 cost =  [3.14971253e+11]
Training with alpha =  1  and lambda =  0.001
 Skipped
 cost =  [3.14617462e+11]
Training with alpha =  1  and lambda =  0.0001
 Skipped
 cost =  [3.14582084e+11]
Training with alpha =  0.1  and lambda =  0.01
 cost =  [0.00841447]
Training with alpha =  0.1  and lambda =  0.001
 cost =  [0.00573881]
Training with alpha =  0.1  and lambda =  0.0001
 cost =  [0.00547118]
Training with alpha =  0.01  and lambda =  0.01
 cost =  [0.0130188]
Training with alpha =  0.01  and lambda =  0.001
 cost =  [0.0121306]
Training with alpha =  0.01  and lambda =  0.0001
 cost =  [0.01204177]


In [615]:
# Use validation set to find optimal alpha and lambda:

for a in range(0, len(alpha_list)):
    for l in range(0, len(lam_list)):
        sumSqrError = 0
        sumSqrParam = 0
        for i in range(0, size_val):
            sumSqrError += (Xval[i] @ param_all[:,a,l] - Yval[i]) ** 2
        for j in range(1, num_features):
            sumSqrParam += param[j] ** 2
        cost = (1/(2*size_val)) * sumSqrError + lam_list[l] * sumSqrParam
        
        if cost < cost_opt:
            cost_opt = cost
            alpha_opt_index = a
            lam_opt_index = l

param = param_all[:, alpha_opt_index, lam_opt_index]
print("Validation set has cost = ", cost_opt)
print("alpha = ", alpha_list[alpha_opt_index])
print("lambda = ", lam_list[lam_opt_index])
#print("Parameters: ", param)

Validation set has cost =  [0.006195]
alpha =  0.1
lambda =  0.0001


The test set examines the results of learning. 

In [616]:
# Test on the test set:

print("Test set results:")

data = numpy.zeros((numpy.size(Xtest,0), numpy.size(Xtest,1)))
label = numpy.zeros((numpy.size(Ytest)))

sumSqrError = 0
for i in range(0, size_test):
    sumSqrError += (Xtest[i] @ param - Ytest[i]) ** 2
    
    # Visualize result:
    # Reverse normalization: 
    label[i] = Ytest[i] * numpy.std(Y) + numpy.mean(Y) 
    data[i,0] = 1.0
    for j in range(1, num_features):
        data[i,j] = Xtest[i,j] * X_std[0,j-1] + X_mean[0,j-1]  
    # Output predicted stock prices of test set:
    print("Prediction = ", round(data[i]@param, 2), ", Y = ", round(label[i], 2))

# Cost calculation:
sumSqrParam = 0
for j in range(1, num_features):
    sumSqrParam += param[j] ** 2

cost = (1/(2*size_test)) * sumSqrError + lam_list[lam_opt_index] * sumSqrParam

print("Test set has cost = ", cost, end='\n')

Test set results:
Prediction =  308.93 , Y =  317.42
Prediction =  243.52 , Y =  253.89
Prediction =  296.67 , Y =  303.95
Prediction =  255.88 , Y =  264.55
Prediction =  272.42 , Y =  330.0
Prediction =  213.71 , Y =  220.33
Prediction =  224.72 , Y =  235.54
Prediction =  1434.35 , Y =  1794.99
Prediction =  271.07 , Y =  266.77
Prediction =  246.67 , Y =  257.82
Prediction =  253.45 , Y =  256.14
Prediction =  280.95 , Y =  279.91
Prediction =  254.02 , Y =  258.33
Prediction =  253.58 , Y =  266.07
Prediction =  771.47 , Y =  796.4
Prediction =  601.66 , Y =  653.0
Prediction =  220.22 , Y =  223.22
Prediction =  347.76 , Y =  349.95
Prediction =  301.49 , Y =  304.0
Prediction =  314.64 , Y =  321.53
Prediction =  343.76 , Y =  357.6
Prediction =  509.77 , Y =  516.65
Prediction =  348.07 , Y =  354.48
Prediction =  246.35 , Y =  257.21
Prediction =  310.15 , Y =  311.85
Prediction =  336.7 , Y =  352.3
Prediction =  569.84 , Y =  650.88
Prediction =  811.06 , Y =  824.75
Predict

In [618]:
# Predict tomorrow's price:
recent = numpy.zeros((1, num_features), dtype=float) 
    # The most recent "lag" amount of prices
    
for j in range(0, num_features-1):
    recent[0,j] = X[-1,j]

recent[0,0] = 1.0
recent[0,-1] = Y[-1] # The last feature is the current price
print(recent)

prediction = recent @ param
print("On July 27th 2020, TSLA highest price will be ", prediction)

[[1.00000000e+00 1.37779004e+03 1.42950000e+03 1.41726001e+03
  1.40856006e+03 1.54892004e+03 1.79498999e+03 1.59000000e+03
  1.55000000e+03 1.53170996e+03 1.53751001e+03 1.65000000e+03
  1.67500000e+03 1.62642004e+03 1.68900000e+03 1.46500000e+03]]
July 27th 2020's price will be  [1543.48248516]
